In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import multiprocessing as mp
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch.optim import Adam
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
os.environ["CUDA_VISIBLE_DEVICES"] = '2'

In [2]:
trained_model_path = '/workdir/security/home/junjiehuang2468/paper/trained_models_weight/ember/'
best_trained_model = '2022-01-18 14:55/2w_epoch:13_test_acc:0.896058.pt'
data_path = "/workdir/security/home/junjiehuang2468/paper/data/ember2018/"
train_data_path = data_path + "malwares/" 
train_label_path = data_path + "train_labels.csv" 

In [3]:
CUDA = True if torch.cuda.is_available() else False
NUM_WORKERS = 24
BATCH_SIZE = 100
LEAVE_BIT_NUMBER = 20000
KERNEL_SIZE = 500

In [4]:
LEAVE_BIT_NUMBER_DICT = {10000:'1w',20000:'2w'}

In [5]:
trainset = pd.read_csv(data_path + 'train_dataset.csv')
validset = pd.read_csv(data_path + 'valid_dataset.csv')
validset = validset.iloc[np.argwhere(validset['labels'].values == 1).squeeze(),:]

In [6]:
class ExeDataset(Dataset): # ExeDataset不是我取的，要記得改
    def __init__(self, malware_names, data_path, labels, leave_bit_num):
        self.malware_names = malware_names
        self.data_path = data_path
        self.labels = labels
        self.leave_bit_num = leave_bit_num

    def __len__(self):
        return len(self.malware_names)

    def __getitem__(self, idx):
        with open(self.data_path + self.malware_names[idx] + '.txt','rb') as fp:
            data = [bit+1 for bit in fp.read()[:self.leave_bit_num]]
            padding = [0]*(self.leave_bit_num-len(data))
            data = data + padding

        return np.array(data), np.array([self.labels[idx]])

In [7]:
train_dataset = ExeDataset(
    trainset["id"].tolist(), 
    train_data_path, 
    trainset["labels"].tolist(), 
    LEAVE_BIT_NUMBER
)
valid_dataset = ExeDataset(
    validset["id"].tolist(), 
    train_data_path, 
    validset["labels"].tolist(), 
    LEAVE_BIT_NUMBER
)

In [8]:
trainloader = DataLoader(
    dataset = train_dataset,
    batch_size = BATCH_SIZE,
    shuffle = False,
    num_workers = NUM_WORKERS,
    pin_memory = True
)
validloader = DataLoader(
    dataset = valid_dataset,
    batch_size = BATCH_SIZE,
    shuffle = False,
    num_workers = NUM_WORKERS,
    pin_memory = True
)

In [9]:
def mp_func(i,inpu,te,gr):
    cum_grad = 0
    for j,(inp,g,t) in enumerate(zip(inpu,gr,te)):
        if inp != 0: continue
        max_idx = np.argmin(g).tolist()
        org_max_idx = np.argmax(t).tolist()
        cum_grad += np.mean(g)
        if g[max_idx] > 0: continue
        te[j][org_max_idx] = 0
        te[j][max_idx] = 1
    return [i,te,cum_grad]

In [10]:
class Model(nn.Module):
    def __init__(self, data_length = 2e6, kernel_size = 500):
        super().__init__()
        self.embedding = nn.Embedding(257, 8, padding_idx=0)
        self.conv_layer_1 = nn.Conv1d(4, 128, kernel_size, stride = kernel_size, bias = True)
        # self.bn_1 = nn.BatchNorm1d(128)
        self.conv_layer_2 = nn.Conv1d(4, 128, kernel_size, stride = kernel_size, bias = True)
        self.pool_layer_2 = nn.MaxPool1d(data_length//kernel_size)
        self.fc_layer_3 = nn.Linear(128, 128)
        self.fc_layer_4 = nn.Linear(128, 2)

    def forward(self, input_, loss_fn):
        temp = F.one_hot(input_,num_classes=257).float()
        temp.requires_grad = True
        temp.retain_grad()
        batch_acc = []
        batch_grad = []
        for _ in range(9):
            x = temp @ self.embedding.weight
            x = torch.transpose(x, -1, -2)

            x_conv_1 = self.conv_layer_1(x[:,:4,:])
            x_conv_2 = torch.sigmoid(self.conv_layer_2(x[:,4:,:]))

            x = x_conv_1*x_conv_2
            del x_conv_1,x_conv_2
            x = self.pool_layer_2(x).squeeze()
            
            x = self.fc_layer_3(x)
            x = self.fc_layer_4(x)
            
            acc = torch.argmax(torch.softmax(x,dim=-1),dim=-1).float().mean()
            batch_acc.append(acc.tolist())
            
            mislead_labels = torch.cat((torch.ones(len(x)).reshape(-1,1),torch.zeros(len(x)).reshape(-1,1)),dim=1).cuda()
            loss = loss_fn(x,mislead_labels)
            loss.backward()
            
            data = [(i,inpu,te,gr) for i,(inpu,te,gr) in enumerate(zip(
                input_.detach().cpu().numpy(),
                temp.detach().cpu().numpy(),
                temp.grad.detach().cpu().numpy()
            ))]
            with mp.Pool(processes=30 if len(data) > 30 else len(data)) as pool:
                results = pool.starmap(mp_func,data)
            results = sorted(results,key = lambda x: x[0])
            for i in range(len(temp)):
                temp.data[i] = torch.tensor(results[i][1], dtype=torch.float, requires_grad=True).cuda()
            batch_grad.append(np.mean([result[2] for result in results]))
            
            temp.grad.zero_()
            
        x = temp @ self.embedding.weight
        x = torch.transpose(x, -1, -2)

        x_conv_1 = self.conv_layer_1(x[:,:4,:])
        x_conv_2 = torch.sigmoid(self.conv_layer_2(x[:,4:,:]))

        x = x_conv_1*x_conv_2
        del x_conv_1,x_conv_2
        x = self.pool_layer_2(x).squeeze()

        x = self.fc_layer_3(x)
        x = self.fc_layer_4(x)
        return x.detach().cpu().numpy(),batch_acc,batch_grad

In [11]:
model = Model(data_length=LEAVE_BIT_NUMBER,kernel_size=KERNEL_SIZE)

ce_loss = nn.CrossEntropyLoss()
optim = Adam(model.parameters())

model = model.cuda() if CUDA else model
ce_loss = ce_loss.cuda() if CUDA else ce_less

In [12]:
model.load_state_dict(torch.load(trained_model_path + best_trained_model))

<All keys matched successfully>

In [ ]:
model.eval()
acc = []
preds = []
labels = []
total_batch_acc = []
total_batch_grad = []
bar = tqdm(validloader)
for step, (batch_data,batch_label) in enumerate(bar):
    batch_data = batch_data.cuda() if CUDA else batch_data
    batch_label = batch_label.cuda() if CUDA else batch_label
    batch_label = batch_label.squeeze()
    temp = torch.zeros((len(batch_label),2))
    for idx,target in enumerate(batch_label.squeeze()): temp[idx,target] = 1
    temp_label = temp.cuda() if CUDA else temp

    pred,batch_acc,batch_grad = model(batch_data,ce_loss)
    
    pred = np.argmax(pred,1)
    batch_label = batch_label.cpu().data.numpy()
    
    preds.extend(pred.tolist())
    labels.extend(batch_label.tolist())
    
    temp_acc = (batch_label == pred).mean()
    acc.append(temp_acc)
    total_batch_acc.append(batch_acc + [temp_acc])
    total_batch_grad.append([0] + batch_grad)
    total_batch_acc_str = '[' + ' '.join(map(lambda x: '%.7f'%x,np.mean(total_batch_acc,axis=0))) + ']'
    total_batch_grad_str = '[' + ' '.join(map(lambda x: '%.7f'%x,np.mean(total_batch_grad,axis=0))) + ']'
    print(total_batch_acc_str)
    print(total_batch_grad_str)
    print(f"test：{temp_acc:4f}, test mean: {np.mean(acc):4f}")

  0%|          | 1/600 [05:21<53:24:45, 321.01s/it]

[0.8700000 0.1200000 0.2900000 0.1000000 0.3500000 0.1500000 0.2900000 0.1600000 0.3900000 0.1000000]
[0.0000000 -0.0011380 0.0008753 0.0023526 -0.0006273 -0.0010835 0.0000943 -0.0011837 0.0017727 -0.0020172]
test：0.100000, test mean: 0.100000


  0%|          | 2/600 [10:21<51:21:06, 309.14s/it]

[0.8650000 0.1100000 0.2350000 0.1050000 0.3000000 0.1500000 0.2700000 0.1300000 0.3400000 0.1200000]
[0.0000000 -0.0010860 0.0006515 0.0012957 -0.0005290 -0.0013705 0.0002178 -0.0010544 0.0012077 -0.0018794]
test：0.140000, test mean: 0.120000


  0%|          | 3/600 [15:34<51:30:12, 310.57s/it]

...

[0.8893846 0.1623077 0.2763077 0.1621538 0.3757949 0.1942051 0.3099487 0.1589744 0.3958974 0.1875897]
[0.0000000 -0.0019508 0.0006841 -0.0006762 -0.0003797 -0.0016480 0.0001554 -0.0014329 0.0001402 -0.0016402]
test：0.190000, test mean: 0.187590


 33%|███▎      | 196/600 [17:52:42<33:50:04, 301.50s/it]

[0.8892347 0.1625510 0.2764286 0.1623980 0.3760714 0.1945918 0.3102041 0.1591326 0.3960204 0.1878571]
[0.0000000 -0.0019504 0.0006827 -0.0006754 -0.0003832 -0.0016468 0.0001543 -0.0014326 0.0001464 -0.0016368]
test：0.240000, test mean: 0.187857


 33%|███▎      | 197/600 [17:57:41<33:38:43, 300.55s/it]

[0.8892893 0.1627411 0.2765990 0.1627411 0.3762436 0.1947208 0.3106091 0.1591878 0.3962436 0.1881218]
[0.0000000 -0.0019536 0.0006834 -0.0006784 -0.0003853 -0.0016474 0.0001546 -0.0014298 0.0001448 -0.0016387]
test：0.240000, test mean: 0.188122
